## Introducción

Importamos librerías y bases de datos que vamos a usar.

Pendientes para este código:

1. Hacer el mismo trabajo que hicimos con la base de 2 ambientes para la base de 3 ambientes.
2. Incluir otras variables además de salarios RIPTE

In [ ]:
# Instalar librerías en las versiones utilizadas en este trabajo
!pip install -r requirements.txt

In [31]:
# Importamos librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [32]:
# Cargamos bases
ruta_base = os.path.join(os.pardir, "Bases")
df_2amb = pd.read_excel(ruta_base + "/Bases IDECBA/2amb.xlsx")
df_3amb = pd.read_excel(ruta_base + "/Bases IDECBA/3amb.xlsx")
df_ipc = pd.read_excel(ruta_base + "/Bases auxiliares/IPC_Trimestral.xlsx")
df_ripte = pd.read_excel(ruta_base + "/Bases auxiliares/base_ripte.xlsx")


## Base auxiliar IPC

Trabajamos con la base del IPC trimestral para calcular la variación de cada trimestre con respecto al 1T 2018, el primero de nuestra base de precios de alquileres.

Esto nos va a servir para después deflactar los valores de las otras series (alquileres 2 y 3 ambientes y salarios RIPTE).

In [ ]:
# Calculamos la variación del IPC para cada trimestre desde el primer trimestre
# Guardamos el valor base que voy a usar en el loop
b1T18 = df_ipc['IPC_trim_b1t16'][0]

# Creamos una columna nueva con valores nulos
df_ipc['inf_b1t18'] = np.nan

# Guardamos la columna para usar en el loop
column = 'IPC_trim_b1t16'

# Calculamos la variación respecto al valor base para cada fila
for row in range(1, len(df_ipc['Trimestre'])):
  df_ipc['inf_b1t18'][row] = (df_ipc[column][row] - b1T18)/b1T18

## Base salarios RIPTE

Trabajamos con la base de salarios RIPTE trimestral para obtener sus valores a precios del 1T 2018, usando la variación de la inflación que calculamos antes con la base IPC. Luego, convertimos esos valores a base 100 = 1T 2018.

In [ ]:
# Calculamos el salario a valores de 1T 2018 para cada trimestre
# Creamos una columna con valores nulos
df_ripte['sal_ajustado_b1t18'] = np.nan

# Seleccionamos la columna que voy a usar en el loop
column = 'inf_b1t18'

# Hacemos un loop para calcular los salarios a valores de 1T 2018 para cada trimestre
for fila in range(1, len(df_ripte['Trimestre'])): #itera filas
    precio_nominal = df_ripte['salario_ripte'][fila]
    df_ripte['sal_ajustado_b1t18'][fila] = precio_nominal/(1+df_ipc[column][fila]) # el index de las filas de df_ipc y df_ripte es el mismo

In [ ]:
# Seteamos el valor correspondiente al 1T 2018 = valor original, dado que es el valor de referencia
df_ripte['sal_ajustado_b1t18'][0] = df_ripte['salario_ripte'][0]

In [ ]:
# Calculamos los salarios a valores de 1T 2018 con base 100 = 1T 2018
# Creamos una columna nueva
df_ripte['sal_ajustado_b100_1t18'] = np.nan

# Guardamos el valor de referencia que voy a usar en el loop
sal_1t18 = df_ripte['sal_ajustado_b1t18'][0]

for fila in range(1, len(df_ripte['Trimestre'])): #itera filas
  precio = df_ripte['sal_ajustado_b1t18'][fila]
  df_ripte['sal_ajustado_b100_1t18'][fila] = precio*100/sal_1t18

In [ ]:
# Seteamos el salario del 1T 2018 = 100
df_ripte['sal_ajustado_b100_1t18'][0] = 100

## Base 2 ambientes

Empieza el trabajo con las bases del GCBA, en particular con la de 2 ambientes. Transponemos los datos para que quede una comuna por columna y que cada fila sea un trimestre desde 1T 2018 hasta 1T 2024, como el resto de las bases con las que trabajamos. Calculamos los precios para cada columna a precios del 1T 2018 (primera fila de cada columna). Luego, transformamos esos valores a base 100 = 1T 2018 para que sean comparables entre sí y con la serie de salarios.

In [ ]:
# Convertimos nulos a NaN
df_2amb = df_2amb.replace({"///": np.nan})

In [ ]:
# Transponemos base 2amb, reseteamos index y cambiamos el nombre de las columnas
df2_trans = df_2amb.transpose()
df2_trans = df2_trans.reset_index()
df2_trans['index'][0] = 'Trimestre'
df2_trans.columns = df2_trans.iloc[0]
df2_trans = df2_trans.drop(0)
df2_trans = df2_trans.reset_index(drop=True)

In [40]:
# Creamos df nuevo para calcular los precios indexados al 1T 2018
# Copiamos el df de 2 ambiente, dado que vamos a usar las mismas columnas
df_2amb_index_1t18 = df2_trans

In [41]:
# Seleccionamos columna para usar en el loop
column = 'inf_b1t18'

In [ ]:
# Calculamos el precio a precios de 1T 2018 para cada comuna y trimestre
for comuna in range(1, 16): #itera columnas (comunas)
  for fila in range(1, len(df_2amb_index_1t18['Trimestre'])): #itera filas
    precio_nominal = df_2amb_index_1t18[comuna][fila]
    df_2amb_index_1t18[comuna][fila] = precio_nominal/(1+df_ipc[column][fila]) # el index de las filas de df_ipc y df_2amb_index_1t18 es el mismo

In [ ]:
# Calculamos lo mismo para la columna "Total"
for fila in range(1, len(df_2amb_index_1t18['Trimestre'])): #itera filas
    precio_nominal = df_2amb_index_1t18['Total'][fila]
    df_2amb_index_1t18['Total'][fila] = precio_nominal/(1+df_ipc[column][fila]) # el index de las filas de df_ipc y df_2amb_index_1t18 es el mismo

In [ ]:
df_2amb_index_1t18

In [45]:
# Creamos un df nuevo para base 100 = 1T 2018
df_2amb_b100_1t18 = df_2amb_index_1t18

In [ ]:
# Calculamos el precio a precios de 1T 2018 con base 100 = 1T 2018
for comuna in range(1, 16): #itera columnas (comunas)
  precio_1t18_comuna = df_2amb_b100_1t18[comuna][0] # Guardo el valor de referencia para cada columna
  for fila in range(1, len(df_2amb_b100_1t18['Trimestre'])): #itera filas
    precio = df_2amb_b100_1t18[comuna][fila]
    df_2amb_b100_1t18[comuna][fila] = precio*100/precio_1t18_comuna

In [ ]:
# Cambiamos los valores de referencia a 100
for comuna in range(1, 16):
  df_2amb_b100_1t18[comuna][0] = 100

In [ ]:
# Hacemos lo mismo que antes pero para la columna "Total"
precio_1t18_tot = df_2amb_b100_1t18['Total'][0] # Guardo el valor de referencia que voy a usar en el loop
for fila in range(1, len(df_2amb_b100_1t18['Trimestre'])): #itera filas
  precio = df_2amb_b100_1t18['Total'][fila]
  df_2amb_b100_1t18['Total'][fila] = precio*100/precio_1t18_tot

In [ ]:
# Cambiamos el valor de referencia a 100
df_2amb_b100_1t18['Total'][0] = 100

In [ ]:
df_2amb_b100_1t18

In [51]:
# Eliminamos comuna 8 porque no hay datos para ningún trimestre
df_2amb_b100_1t18 = df_2amb_b100_1t18.drop(columns=8)

In [52]:
# Definimos zonas
zona_norte = [2, 13, 14]
zona_centro = [1, 3, 5, 6, 7, 11, 12, 15]
zona_sur = [4, 9, 10] #Sacamos comuna 8 porque no tiene valores en la base

In [53]:
# Armamos una columna nueva por zona que sea el promedio de las comunas de la zona
df_2amb_b100_1t18['Zona norte'] = df_2amb_b100_1t18[zona_norte].mean(axis=1)
df_2amb_b100_1t18['Zona centro'] = df_2amb_b100_1t18[zona_centro].mean(axis=1)
df_2amb_b100_1t18['Zona sur'] = df_2amb_b100_1t18[zona_sur].mean(axis=1)

In [ ]:
df_2amb_b100_1t18

## Gráfico 2 ambientes

Definimos variables para el gráfico de 2 ambientes + salarios.

In [55]:
# Definimos variables para el gráfico
x2 = df_2amb_b100_1t18['Trimestre']
ytot2 = df_2amb_b100_1t18['Total']
ynor2 = df_2amb_b100_1t18['Zona norte']
ycen2 = df_2amb_b100_1t18['Zona centro']
ysur2 = df_2amb_b100_1t18['Zona sur']
ysal2 = df_ripte['sal_ajustado_b100_1t18'] # Agregamos una columna de otro df porque las filas son las mismas

In [ ]:
grafico_2amb = plt.figure(figsize=(12, 4.7), layout='constrained')
plt.plot(x2, ytot2, label='Promedio 2amb', linewidth=2.5, color='black', linestyle='--')
plt.plot(x2, ynor2, label='Zona Norte')
plt.plot(x2, ycen2, label='Zona Centro')
plt.plot(x2, ysur2, label='Zona Sur')
plt.plot(x2, ysal2, label = 'Salarios')
plt.legend()

# Base 3 ambientes

Retomamos el mismo procedimiento para la base de 3 ambientes del GCBA. Transponemos los datos para que quede una comuna por columna y que cada fila sea un trimestre desde 1T 2018 hasta 1T 2024, como el resto de las bases con las que trabajamos. Calculamos los precios para cada columna a precios del 1T 2018 (primera fila de cada columna). Luego, transformamos esos valores a base 100 = 1T 2018 para que sean comparables entre sí y con la serie de salarios

In [ ]:
# Convertimos nulos a NaN
df_3amb = df_3amb.replace({"///": np.nan})

In [ ]:
# Transponemos base 3amb, reseteamos index y cambiamos el nombre de las columnas
df3_trans = df_3amb.transpose()
df3_trans = df3_trans.reset_index()
df3_trans['index'][0] = 'Trimestre'
df3_trans.columns = df3_trans.iloc[0]
df3_trans = df3_trans.drop(0)
df3_trans = df3_trans.reset_index(drop=True)

In [59]:
# Creamos df nuevo para calcular los precios indexados al 1T 2018
# Copiamos el df de 3 ambiente, dado que vamos a usar las mismas columnas
df_3amb_index_1t18 = df3_trans

In [ ]:
# Calculamos el precio a precios de 1T 2018 para cada comuna y trimestre
for comuna in range(1, 16): #itera columnas (comunas)
  for fila in range(1, len(df_3amb_index_1t18['Trimestre'])): #itera filas
    precio_nominal = df_3amb_index_1t18[comuna][fila]
    df_3amb_index_1t18[comuna][fila] = precio_nominal/(1+df_ipc[column][fila]) # el index de las filas de df_ipc y df_3amb_index_1t18 es el mismo

In [ ]:
# Calculamos lo mismo para la columna "Total"
for fila in range(1, len(df_3amb_index_1t18['Trimestre'])): #itera filas
    precio_nominal = df_3amb_index_1t18['Total'][fila]
    df_3amb_index_1t18['Total'][fila] = precio_nominal/(1+df_ipc[column][fila]) # el index de las filas de df_ipc y df_3amb_index_1t18 es el mismo

In [ ]:
df_3amb_index_1t18

In [63]:
# Creamos un df nuevo para base 100 = 1T 2018
df_3amb_b100_1t18 = df_3amb_index_1t18

In [ ]:
# Calculamos el precio a precios de 1T 2018 con base 100 = 1T 2018
for comuna in range(1, 16): #itera columnas (comunas)
  precio_1t18_comuna = df_3amb_b100_1t18[comuna][0] # Guardo el valor de referencia para cada columna
  for fila in range(1, len(df_3amb_b100_1t18['Trimestre'])): #itera filas
    precio = df_3amb_b100_1t18[comuna][fila]
    df_3amb_b100_1t18[comuna][fila] = precio*100/precio_1t18_comuna

In [ ]:
# Cambiamos los valores de referencia a 100
for comuna in range(1, 16):
  df_3amb_b100_1t18[comuna][0] = 100

In [ ]:
# Hacemos lo mismo que antes pero para la columna "Total"
precio_1t18_tot_3amb = df_3amb_b100_1t18['Total'][0] # Guardo el valor de referencia que voy a usar en el loop
for fila in range(1, len(df_3amb_b100_1t18['Trimestre'])): #itera filas
  precio = df_3amb_b100_1t18['Total'][fila]
  df_3amb_b100_1t18['Total'][fila] = precio*100/precio_1t18_tot_3amb

In [ ]:
# Cambiamos el valor de referencia a 100
df_3amb_b100_1t18['Total'][0] = 100

In [ ]:
df_3amb_b100_1t18

In [69]:
# Eliminamos comuna 8 y 9 porque no hay datos para ningún trimestre
df_3amb_b100_1t18 = df_3amb_b100_1t18.drop(columns=[8, 9])

In [70]:
# Definimos zonas
zona_norte = [2, 13, 14]
zona_centro = [1, 3, 5, 6, 7, 11, 12, 15]
zona_sur = [4, 10] #Sacamos comunas 8 y 9 porque no tiene valores en la base

In [71]:
# Armamos una columna nueva por zona que sea el promedio de las comunas de la zona
df_3amb_b100_1t18['Zona norte'] = df_3amb_b100_1t18[zona_norte].mean(axis=1)
df_3amb_b100_1t18['Zona centro'] = df_3amb_b100_1t18[zona_centro].mean(axis=1)
df_3amb_b100_1t18['Zona sur'] = df_3amb_b100_1t18[zona_sur].mean(axis=1)

In [ ]:
df_3amb_b100_1t18

# Gráfico 3 ambientes

Procedemos del mismo modo que para el gráfico del dataframe de 2 ambientes, definiendo variables para 3 ambientes + salarios.

In [73]:
# Definimos variables para el gráfico
x3 = df_3amb_b100_1t18['Trimestre']
ytot3 = df_3amb_b100_1t18['Total']
ynor3 = df_3amb_b100_1t18['Zona norte']
ycen3 = df_3amb_b100_1t18['Zona centro']
ysur3 = df_3amb_b100_1t18['Zona sur']
ysal3 = df_ripte['sal_ajustado_b100_1t18'] # Agregamos una columna de otro df porque las filas son las mismas

In [ ]:
grafico_3amb = plt.figure(figsize=(12, 4.7), layout='constrained')
plt.plot(x3, ytot3, label='Promedio 3amb', linewidth=2.5, color='black', linestyle='--')
plt.plot(x3, ynor3, label='Zona Norte')
plt.plot(x3, ycen3, label='Zona Centro')
plt.plot(x3, ysur3, label='Zona Sur')
plt.plot(x3, ysal3, label = 'Salarios')
plt.legend()